In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from mvf_bto.data_loading import load_data
from mvf_bto.constants import * 
from mvf_bto.models.baseline_lstm import BaselineLSTM
from mvf_bto.preprocessing import create_discharge_inputs,create_charge_inputs
# from data_loading import load_data
# from constants import *
# from models.baseline_lstm import BaselineLSTM
# from preprocessing import create_discharge_inputs

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError

import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go

## Loading Data

In [5]:
# data_path = "/Users/anoushkabhutani/PycharmProjects/10701-mvf-bto/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"
data_path = "/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto-backup/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"

In [6]:
data = load_data(file_path=data_path, num_cells=9)

100%|██████████| 9/9 [01:22<00:00,  9.12s/it]


## Preprocessing to create model inputs and targets

In [8]:
train_split = 0.7
test_split = 0.2
# by default uses validation_split = 1 - (train_split + test_split)
print(REFERENCE_CHARGE_CAPACITIES)
print(np.where(REFERENCE_CHARGE_CAPACITIES==0.1)[0][0])
print(np.where(REFERENCE_CHARGE_CAPACITIES==0.8)[0][0])


[0.    0.005 0.01  0.015 0.02  0.025 0.03  0.035 0.04  0.045 0.05  0.055
 0.06  0.065 0.07  0.075 0.08  0.085 0.09  0.095 0.1   0.1   0.2   0.3
 0.4   0.5   0.6   0.7   0.8   0.8   0.805 0.81  0.815 0.82  0.825 0.83
 0.835 0.84  0.845 0.85  0.85  0.86  0.87  0.88  0.89  0.9   0.91  0.92
 0.93  0.94  0.95  0.96  0.97  0.98  0.98  1.   ]
20
28


In [38]:
# datasets = create_discharge_inputs(data, train_split, test_split)
datasets,train_cell,test_cell,validation_cell = create_charge_inputs(data, train_split, test_split)

100%|██████████| 869/869 [00:04<00:00, 199.83it/s]


In [39]:
print(train_cell,test_cell,validation_cell)

['b1c2', 'b1c8', 'b1c5', 'b1c1', 'b1c3', 'b1c4'] ['b1c6'] ['b1c0', 'b1c7']


## Train Model

In [40]:
window_length = datasets["X_train"].shape[1]
n_features = datasets["X_train"].shape[2]
batch_size=datasets["batch_size"]
batch_input_shape = (datasets["batch_size"], window_length, n_features)
n_outputs = datasets["y_train"].shape[1]
print(window_length, n_features, batch_input_shape, batch_size, n_outputs,datasets["X_train"].shape[0]//batch_size)

3 4 (53, 3, 4) 53 2 4798


In [27]:
# import tensorflow as tf
# class BaselineLSTM(tf.keras.Model):
#     def __init__(
#             self,
#             batch_input_shape,
#             n_outputs,
#             lstm1_units=32,
#             lstm2_units=16,
#             lstm3_units=16,
#             dense1_units=16,
#             dense2_units=8,
#     ):
#         super().__init__()
#         self.lstm_1 = tf.keras.layers.LSTM(
#             units=lstm1_units,
#             return_sequences=True,
#             stateful=True,
#             batch_input_shape=batch_input_shape,
#         )
#         self.lstm_2 = tf.keras.layers.LSTM(
#             units=lstm2_units,
#             return_sequences=False,
#             stateful=False,
#         )
#         # self.lstm_3 = tf.keras.layers.LSTM(
#         #     units=lstm3_units,
#         #     return_sequences=False,
#         #     stateful=False,
#         # )        
#         self.dense_1 = tf.keras.layers.Dense(units=dense1_units, activation="relu")
#         self.dense_2 = tf.keras.layers.Dense(units=dense2_units)
#         self.output_layer = tf.keras.layers.Dense(units=n_outputs)

#     def call(self, inputs):
#         x = self.lstm_1(inputs)
#         x = self.lstm_2(x)
#         # x = self.lstm_3(x)
#         x = self.dense_1(x)
#         x = self.dense_2(x)
#         return self.output_layer(x)

In [28]:
model = BaselineLSTM(batch_input_shape=batch_input_shape, n_outputs=n_outputs)


In [29]:
model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
model.build(input_shape=batch_input_shape)

In [30]:
model.summary()

Model: "baseline_lstm_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               multiple                  4736      
                                                                 
 lstm_7 (LSTM)               multiple                  3136      
                                                                 
 dense_9 (Dense)             multiple                  272       
                                                                 
 dense_10 (Dense)            multiple                  136       
                                                                 
 dense_11 (Dense)            multiple                  18        
                                                                 
 reshape_3 (Reshape)         multiple                  0         
                                                                 
Total params: 8,298
Trainable params: 8,298
Non-tra

In [41]:
skip=100
fig = go.Figure()
pallete = plotly.colors.qualitative.Dark24*(len(datasets["X_train"])//skip)

for i in range(0, len(datasets["X_train"]), batch_size * skip):

    df_true = pd.DataFrame(datasets["y_train"][i : i + batch_size])
    
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
            showlegend=True,
            mode="lines+markers",
            name = f"Curve {i//batch_size+1}",
            line_color=pallete[i//skip]
        )
    )

fig.update_yaxes(title="Voltage [V]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")

In [42]:
model = BaselineLSTM(batch_input_shape=batch_input_shape, n_outputs=n_outputs)
model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])

es = EarlyStopping(
    monitor="val_mean_squared_error",
    min_delta=0,
    patience=10,
    verbose=1,
    mode="auto",
    restore_best_weights=True,
)

history = model.fit(
    datasets["X_train"],
    datasets["y_train"],
    validation_data=(datasets["X_val"], datasets["y_val"]),
    epochs=50,
    batch_size=datasets["batch_size"],
    shuffle=False,
    callbacks=[es],
    verbose=1,
)

Epoch 1/50
4798/4798 [==============================] - 28s 5ms/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.0095 - val_mean_squared_error: 0.0095
Epoch 2/50
4798/4798 [==============================] - 23s 5ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0096 - val_mean_squared_error: 0.0096
Epoch 3/50
4798/4798 [==============================] - 23s 5ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0096 - val_mean_squared_error: 0.0096
Epoch 4/50
4798/4798 [==============================] - 23s 5ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0096 - val_mean_squared_error: 0.0096
Epoch 5/50
4798/4798 [==============================] - 23s 5ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0096 - val_mean_squared_error: 0.0096
Epoch 6/50
4798/4798 [==============================] - 22s 5ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0096 - val_mean_squared_error: 0.0096
Epoch 7/50

In [43]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.linspace(1, 50),
        y=history.history["loss"],
        showlegend=False,
        mode="markers+lines",
    )
)
fig.update_xaxes(title="Epochs")
fig.update_yaxes(title="Loss (MSE)")

KeyboardInterrupt: 

## Parity Plot of Training Error

In [100]:
# random plotting traing error at some interval = skip to not make the plot rendering too slow
batch_size = datasets["batch_size"]
skip = 70

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1.5], y=[0, 1.5], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_train"][i : i + batch_size], verbose=0, batch_size=batch_size))
    df_train = pd.DataFrame(datasets["y_train"][i : i + batch_size])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")

In [101]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1.5], y=[0, 1.5], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_train"][i : i + batch_size], verbose=0,batch_size=batch_size))
    df_train = pd.DataFrame(datasets["y_train"][i : i + batch_size])
    fig.add_trace(
        go.Scatter(
            x=df_pred[1].values,
            y=df_train[1].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Temperature Target")
fig.update_xaxes(title="Normalized Temperature Prediction")

## Parity Plot of Test Error

In [103]:
skip = 20

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1.5], y=[0, 1.5], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_test"]), batch_size*skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0,batch_size=batch_size))
    df_train = pd.DataFrame(datasets["y_test"][i : i + batch_size])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")

In [104]:
print(len(datasets["X_test"]))
print(batch_size)

59307
53


In [106]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0, batch_size=batch_size))
    df_train = pd.DataFrame(datasets["y_test"][i : i + batch_size])
    fig.add_trace(
        go.Scatter(
            x=df_pred[1].values,
            y=df_train[1].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")

## True vs Predicted Traces (Test Set)

In [111]:
skip = 20

pallete = plotly.colors.qualitative.Dark24*(len(datasets["X_test"])//skip)

fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0, batch_size=batch_size))
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size])
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
            showlegend=True,
            mode="markers",
            name = f"Predicted Curve {i+1}",
            marker_color=pallete[i//skip]
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
            showlegend=True,
            mode="lines",
            name = f"True Curve {i+1}",
            line_color=pallete[i//skip]
        )
    )

fig.update_yaxes(title="Voltage [V]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")

In [ ]:
fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0))
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size])
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_pred[1].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
            showlegend=True,
            mode="markers",
            name = f"Predicted Curve {i+1}",
            marker_color=pallete[i//skip]
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_true[1].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
            showlegend=True,
            mode="lines",
            name = f"True Curve {i+1}",
            line_color=pallete[i//skip]
        )
    )

fig.update_yaxes(title="Temperature [°C]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")